In [3]:
import sqlalchemy 
from dotenv import load_dotenv
from os import getenv
import pandas as pd
import sqlalchemy as sa
import plotly.express as px
import seaborn as sns
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [4]:
load_dotenv()

False

In [5]:


__host = getenv("host")
__port = getenv("port")
__username = getenv("username")
__password = getenv("password")
__database_name = getenv("database_name")

host = __host
port = __port
username = __username
password = __password
database_name = __database_name

df = pd.read_sql('SELECT * FROM bike_rentals_project', con = 'mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}')
df

ValueError: invalid literal for int() with base 10: '{port}'

In [ ]:
df['cnt']

## Highlighting Target Vector

In [ ]:
px.histogram(df, x='cnt')

In [ ]:
(round(df['cnt'].median(), 2))

4548.0

In [ ]:
(round(df['cnt'].mean(), 2))

4504.35

In [ ]:
(round(df['cnt'].mode(), 2))

0     1096
1     1162
2     1685
3     1977
4     2077
5     2424
6     2425
7     3068
8     3214
9     3351
10    3784
11    3974
12    4073
13    4098
14    4195
15    4274
16    4401
17    4459
18    4649
19    4758
20    5115
21    5119
22    5191
23    5202
24    5260
25    5312
26    5409
27    5698
28    5847
29    6043
30    6536
31    6591
32    6824
33    6883
34    7534
Name: cnt, dtype: int64

In [6]:
# Let's visualize the relationship between our target and the rest of our features
sns.pairplot(df, y_vars=['cnt'])

NameError: name 'df' is not defined

### Dropping columns to improve model training and prediction performance

In [ ]:
df.drop(axis = 1, columns = ['instant', 'dteday', 'casual', 'registered'], inplace = True)

df.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,temp_fahrenheit
0,spring,0,1,0,6,0,2,14.11,18.18,80.58,10.75,985,57.40
1,spring,0,1,0,0,0,2,14.90,17.69,69.61,16.65,801,58.82
2,spring,0,1,0,1,1,1,8.05,9.47,43.73,16.64,1349,46.49
3,spring,0,1,0,2,1,1,8.20,10.61,59.04,10.74,1562,46.76
4,spring,0,1,0,3,1,1,9.31,11.46,43.70,12.52,1600,48.76


### Establish the baseline:

In [ ]:
baseline = [df['cnt'].mean()] * ( len(df))
print(f'Baseline is: {round(baseline[0], 2)}')

# Check:
len(baseline)
len(df)



Baseline is: 4504.35


731

### Our target variable is:

Count of total rental bikes per day [cnt]


In [ ]:
X = df

y = df['cnt']

### Splitting our data into training and test sets

In [ ]:

X_train, v, y_train, y = train_test_split(X,y, test_size=.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(v,y, test_size=.5, random_state=42)


### Make a pipeline

In [ ]:

model_rf = make_pipeline(

    OrdinalEncoder(),
    RandomForestRegressor(random_state=42)
)


model_rf.fit(X_train, y_train)


Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['season'],
                                mapping=[{'col': 'season',
                                          'data_type': dtype('O'),
                                          'mapping': winter    1
fall      2
summer    3
spring    4
NaN      -2
dtype: int64}])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [ ]:
train_base = [df['cnt'].mean()] * len(y_train)
print(train_base)


[4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025, 4504.3488372093025,

In [ ]:
print(f'Training Baseline Error: {round (mean_absolute_error(y_train, train_base),2)}')
print(f'Training Error: {round (mean_absolute_error(y_train, model_rf.predict(X_train)),2)}')
print(f'Testing Error: {round (mean_absolute_error(y_test, model_rf.predict(X_test)),2)}')


Training Baseline Error: 1550.33
Training Error: 4.43
Testing Error: 8.07


In [ ]:
print(r2_score(y_train, model_rf.predict(X_train))) 
print(r2_score(y_test, model_rf.predict(X_test))) 


0.9999805830051647
0.9999700367461063


In [ ]:
baseline = [df['cnt'].mean()] * len(df)
print(f'Baseline is: {round(baseline[0], 2)}')

Baseline is: 4504.35


## Feature Selection - choosing a subset of column features from the dataset to create a more focused and efficient model.

In [ ]:
# 3-D Matrix, make predictions based off of:
x = df[['cnt', 'mnth', 'temp', 'season']]

# 1-D Target Vector
y = df['cnt']

## Now, we are spliting into training and testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.2, random_state=42)

#### Make a pipeline:

In [ ]:
model_rf = make_pipeline(
    OrdinalEncoder(),
    RandomForestRegressor(random_state=42)
)
model_rf.fit(x_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['season'],
                                mapping=[{'col': 'season',
                                          'data_type': dtype('O'),
                                          'mapping': winter    1
fall      2
summer    3
spring    4
NaN      -2
dtype: int64}])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

feature matrix and actual values for the target vector for the 'model_rf.fit(x_train, y_train)'

In [ ]:
train_base = [df['cnt'].mean()] * len(y_train) # baseline being set

In [ ]:
mean_absolute_error(y_train, train_base) # plus/minus y predict/

1550.3264176489329

In [ ]:
mean_absolute_error(y_train, model_rf.predict(x_train)) # .predict predict the values of the target 

3.336523972602742

In [ ]:
mean_absolute_error(y_test, model_rf.predict(x_test))

10.057414965986368

In [ ]:
mean_absolute_error(y_test, model_rf.predict(x_test))

10.057414965986368

In [ ]:
r2_score(y_train, model_rf.predict(x_train))

0.9999867844391492

In [ ]:
r2_score(y_test, model_rf.predict(x_test))

0.9995819040152059